In [27]:
SEASON = '1992-93'

In [28]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Michael Jordan,CHI,SG,132.687921,29,78,78,39.3,12.7,25.7,...,34.7,12.0,5.2,17.2,0.270,8.7,2.6,11.2,10.2,57.0
2,Hakeem Olajuwon,HOU,C,107.723246,30,82,82,39.5,10.3,19.5,...,28.8,7.9,8.0,15.8,0.234,3.9,3.6,7.5,7.8,55.0
3,Charles Barkley,PHO,PF,102.732040,29,76,76,37.6,9.4,18.1,...,26.9,9.6,4.8,14.4,0.242,6.3,1.4,7.8,7.1,62.0
4,Karl Malone,UTA,PF,95.208015,29,82,82,37.8,9.7,17.6,...,28.4,10.4,5.0,15.4,0.238,6.0,1.4,7.3,7.3,47.0
5,David Robinson,SAS,C,86.244098,27,82,82,39.2,8.2,16.4,...,26.4,6.7,6.4,13.2,0.197,3.6,2.7,6.4,6.8,49.0
6,Brad Daugherty,CLE,C,68.631596,27,71,71,37.9,7.3,12.8,...,21.1,8.8,3.9,12.7,0.226,4.0,0.9,4.9,4.7,54.0
7,John Stockton,UTA,PG,67.051404,30,82,82,34.9,5.3,11.0,...,19.9,7.2,3.4,10.6,0.177,5.1,1.5,6.6,6.2,47.0
8,Patrick Ewing,NYK,C,66.288539,30,81,81,37.1,9.6,19.1,...,29.7,2.6,8.1,10.6,0.170,1.4,2.0,3.4,4.0,60.0
9,Dominique Wilkins,ATL,SF,66.083018,33,71,70,37.3,10.4,22.3,...,31.9,9.1,1.4,10.5,0.191,7.1,-1.7,5.4,4.9,43.0
10,Shaquille O'Neal,ORL,C,60.628759,20,81,81,37.9,9.0,16.1,...,27.0,4.5,6.0,10.4,0.163,2.0,1.5,3.5,4.3,41.0
